In [53]:
import pandas as pd

In [54]:
df = pd.read_csv('train/train.csv')

In [55]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [56]:
df.dropna(inplace=True)

In [57]:
X = df.drop('label',axis=1)

In [58]:
y = df['label']

In [59]:
X.shape

(18285, 4)

In [60]:
y.shape

(18285,)

In [61]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [62]:
import tensorflow as tf

In [63]:
tf.__version__

'2.1.0'

In [64]:

from tensorflow import keras
from keras.layers import Embedding, Dense, LSTM, Dropout,Bidirectional
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [65]:
voc_size= 5000

### One hot representation

In [66]:
titles=X.copy()

In [67]:
titles.reset_index(inplace=True)

In [68]:
### For data Preprocessing

import nltk
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [69]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yyash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [70]:
ps = PorterStemmer()
corpus = []
for i in range(0,len(titles)):
    review = re.sub('[^a-zA-Z]',' ',titles['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

    review = ' '.join(review)
    corpus.append(review)

In [71]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [72]:
one_hot_encoded = [one_hot(sentences,voc_size) for sentences in corpus]

In [73]:
one_hot_encoded

[[2972, 2503, 3843, 3657, 3964, 760, 977, 3567, 2800, 3675],
 [3037, 4556, 3743, 618, 726, 4491, 1251],
 [808, 1068, 1301, 904],
 [4038, 3539, 1202, 3894, 1101, 3424],
 [1541, 726, 1615, 1456, 1410, 4419, 726, 4609, 4466, 4057],
 [2792,
  2211,
  847,
  4195,
  3507,
  402,
  1670,
  3936,
  1739,
  2657,
  684,
  875,
  2479,
  4716,
  1251],
 [2416, 4287, 4291, 1731, 444, 1257, 2301, 1861, 129, 4608, 4448],
 [1642, 672, 2283, 2654, 3241, 1029, 402, 590, 129, 4608, 4448],
 [383, 3513, 669, 1458, 2115, 1155, 4790, 3845, 402, 2377],
 [4308, 2004, 2831, 4318, 3191, 2301, 1144, 42],
 [3155, 2989, 3828, 3329, 248, 4454, 2554, 1075, 2102, 3470, 4104],
 [3894, 1697, 3964, 1155, 402, 3241],
 [2783, 892, 547, 790, 1045, 398, 1542, 461, 4733],
 [1959, 1877, 4687, 1113, 1890, 3470, 1224, 129, 4608, 4448],
 [4817, 1653, 1181, 1990, 4122, 129, 4608, 4448],
 [1151, 4822, 1826, 877, 4865, 3480, 3245, 2014, 4274, 3773],
 [4489, 4133, 4556],
 [627, 2712, 2202, 4679, 402, 248, 1464, 1251],
 [2143, 3938

In [74]:
sent_length = 20

In [75]:
embedded_doc = pad_sequences(one_hot_encoded,padding='pre',maxlen=sent_length)

In [76]:
embedded_doc

array([[   0,    0,    0, ..., 3567, 2800, 3675],
       [   0,    0,    0, ...,  726, 4491, 1251],
       [   0,    0,    0, ..., 1068, 1301,  904],
       ...,
       [   0,    0,    0, ...,  129, 4608, 4448],
       [   0,    0,    0, ..., 3410, 1501,   25],
       [   0,    0,    0, ..., 2000, 4167, 2078]])

In [77]:
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2972,
       2503, 3843, 3657, 3964,  760,  977, 3567, 2800, 3675])

## Model

In [78]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [79]:
len(embedded_doc),y.shape

(18285, (18285,))

In [81]:
import numpy as np
X_final = np.array(embedded_doc)

In [82]:
y_final = np.array(y)

In [83]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [84]:
X_final

array([[   0,    0,    0, ..., 3567, 2800, 3675],
       [   0,    0,    0, ...,  726, 4491, 1251],
       [   0,    0,    0, ..., 1068, 1301,  904],
       ...,
       [   0,    0,    0, ...,  129, 4608, 4448],
       [   0,    0,    0, ..., 3410, 1501,   25],
       [   0,    0,    0, ..., 2000, 4167, 2078]])

In [85]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [86]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

C:\Users\yyash\anaconda3\envs\tensorgpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 8s 669us/step - loss: 0.3077 - accuracy: 0.8525 - val_loss: 0.2035 - val_accuracy: 0.9173
Epoch 2/10
12250/12250 [==============================] - 7s 582us/step - loss: 0.1381 - accuracy: 0.9460 - val_loss: 0.2064 - val_accuracy: 0.9138
Epoch 3/10
12250/12250 [==============================] - 7s 566us/step - loss: 0.0912 - accuracy: 0.9679 - val_loss: 0.2647 - val_accuracy: 0.9059
Epoch 4/10
12250/12250 [==============================] - 7s 557us/step - loss: 0.0636 - accuracy: 0.9781 - val_loss: 0.3122 - val_accuracy: 0.9094
Epoch 5/10
12250/12250 [==============================] - 7s 597us/step - loss: 0.0405 - accuracy: 0.9865 - val_loss: 0.3244 - val_accuracy: 0.9075
Epoch 6/10
12250/12250 [==============================] - 7s 576us/step - loss: 0.0249 - accuracy: 0.9922 - val_loss: 0.4087 - val_accuracy: 0.9032
Epoch 7/10
12250/12250 [==============================] - 7s 58

### Performance Metrics And Accuracy

In [89]:
y_pred = model1.predict_classes(X_test)

In [90]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [94]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [92]:
accuracy_score(y_test,y_pred)

0.8984258492129246

In [93]:
confusion_matrix(y_test,y_pred)

array([[3098,  321],
       [ 292, 2324]], dtype=int64)

In [97]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      3419
           1       0.88      0.89      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

